# introduction notes for 2017.11.09

1. exam answer key posted
2. fixing `s3` permissions / access

## exam answer key posted

the exam answer key has been posted [on github](https://github.com/rzl-ds/gu511_exam/blob/master/2017_midterm_exam_answers.ipynb) and [azure notebooks](https://notebooks.azure.com/rzachlamberty/libraries/gu511-exam)

## fixing `s3` permissions / access

more `s3` permissions fun!

our goal with creating the new `s3` buckets *we* controlled was to remove the permission headaches from you when you're doing your homework submission

our approach was the following:

1. create a new bucket for each user
2. grant full control to that other `aws` account via a canned `acl` (access control list)
3. add the below bucket policy to give both accounts all `s3` permissions on that bucket

```json
{
    "Version": "2012-10-17",
    "Id": "GUID-c2b219e2-c49f-11e7-aa9d-9890969c4c99",
    "Statement": [
        {
            "Sid": "1",
            "Effect": "Allow",
            "Principal": {
                "AWS": [
                    "arn:aws:iam::134461086921:root",
                    "arn:aws:iam::USER_AWS_ID:root"
                ]
            },
            "Action": "s3:*",
            "Resource": [
                "arn:aws:s3:::2017.fall.gu511.GUID/*",
                "arn:aws:s3:::2017.fall.gu511.GUID"
            ]
        }
    ]
}
```

only one problem: files *I* add are all visible to you immediately, but files *you* add are not visible to me by default. 

after some extensive research, it appears that there is one (and only one) solution: you need to update the `acl` value on your files to grant the bucket owner (me) full control.

there are two ways to do this. first, you can re-upload all the files you've added, but with an extra flag:

```bash
aws s3 cp YOUR_FILE s3://2017.fall.gu511.{GUID} --acl bucket-owner-full-control
```

the other option is to add that `acl` directly to an `s3` key without re-uploading:

```bash
aws s3api put-object-acl --bucket s3://2017.fall.gu511.{GUID} --key {KEY_NAME} --acl bucket-owner-full-control
```

if you have several files uploaded already, the following python code should also be sufficient to update the `acl` on each

```python
import boto3

def update_acls(guid):
    session = boto3.session.Session()
    s3 = session.resource('s3')

    bucket = s3.Bucket('2017.fall.gu511.{}'.format(guid))
    for obj in bucket.objects.all():
        resp = obj.Acl().put(ACL='bucket-owner-full-control')
        
        try:
            assert resp['ResponseMetadata']['HTTPStatusCode'] == 200
            print('update of key "{.key}" successful'.format(obj))
        except AssertionError:
            print('updaate of key "{.key}" FAILED'.format(obj))
```

alternatively, you could fix this manually via the web console.

1. go to your bucket at `https://s3.console.aws.amazon.com/s3/buckets/2017.fall.gu511.{GUID}/`
2. for each file you have uploaded
    1. click on the link to that file
    2. open the "Permissions" tab
    3. under "Access for other AWS accounts"
        1. click "+ Add account"
        2. enter `rzl5@georgetown.edu`
        3. check all three boxes "Yes"
        4. click "Save"